In [1]:
!pip install openai together python-dotenv ipywidgets

In [2]:
import together
import os
from tqdm.notebook import tqdm
import smart_open
from dolma.core.paths import glob_path
import random
import json
import copy
import openai
from collections import Counter
from dotenv import load_dotenv
import csv

load_dotenv()

random.seed(42)
base_path = "s3://ai2-llm/pretraining-data/sources/cccc/v2_nc_nd_fix/documents/*/*.gz"
all_paths = list(glob_path(base_path))
random.shuffle(all_paths)

# response = client.chat.completions.create(
#     model="mistralai/Mixtral-8x7B-Instruct-v0.1",
#     messages=[{"role": "user", "content": "tell me about new york"}],
# )
# print(response.choices[0].message.content)

In [48]:
prompt = """
Given the followings HTML snippets enclosed in ```quotes```, respond YES if the Creative Common license mentioned in the snippets refers to to the text content of the web page, otherwise respond NO.

Examples of "NO" include:
- The Creative Common license refers to the images on the page.
- The is another license mentioned on the snippet.
- License is not in an official Creative Common format.
- Mentions that "some", but not "all", of the content is licensed under Creative Common.

Examples of "YES" include:
- Copyright or ©️ is mentioned on the page AND text content is licensed under Creative Common; it is ok if the page is copyrighted if a Creative Common license is also mentioned.
- All "work" or "content" is mentioned as being Creative Common licensed.
- The Creative Commons tag appears on the footer on the page with no extra content.
- The content is in the public domain (i.e., a public domain license in mentioned).

You can use the source URL to help you make an assessment; for example, government and non-profit web pages are more likely to contain creative common licenses. However, DO NOT EXCLUSIVELY rely on the source URL to make a decision.

DO NOT return anything other than "YES" or "NO".

Source URL: {source}
Snippet:
```
{snippet}
```
""".strip()

In [12]:
# Grabbing documents
documents = []
per_snapshot = 10
total_files = 100
sample = 0.01

with tqdm(desc="files", position=0) as tf, tqdm(desc="lines", position=1) as tl:
    j = 0
    for path in all_paths:
        with smart_open.open(path) as f:
            i = 0
            for line in f:
                if random.random() > sample:
                    continue

                documents.append(json.loads(line))
                tl.update(1)
                i += 1
                if i >= per_snapshot:
                    break
        tf.update(1)
        j += 1
        if j >= total_snapshots:
            break

files: 0it [00:00, ?it/s]

lines: 0it [00:00, ?it/s]

KeyboardInterrupt: 

In [44]:
# flushing out the documents
responses = []

In [45]:
client = together.Together(api_key=os.environ.get("TOGETHER_API_KEY"))
# client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

model_name = "meta-llama/Llama-3-70b-chat-hf"
# model_name = "gpt-4o"
print(prompt)

for doc in tqdm(documents):
    extracted_licenses = {
        license_name: "\n...\n".join(s.strip() for s in license_snippet)
        for license_name, license_snippet in doc["metadata"]["attribute_spans"].items()
        if "copyright" not in license_name and isinstance(license_snippet, list)
    }

    labeled_licenses = {}
    for license_name, license_snippet in extracted_licenses.items():
        response = client.chat.completions.create(
            model=model_name,
            temperature=0.2,
            messages=[{"role": "user", "content": prompt.format(snippet=license_snippet)}],
        )
        labeled_licenses[license_name] = {
            "label": response.choices[0].message.content,
            "snippet": license_snippet,
            "model": model_name,
        }

    doc_with_labels = copy.deepcopy(doc)
    doc_with_labels["metadata"]["labeled_licenses"] = labeled_licenses
    # print(json.dumps({'url': doc_with_labels['metadata']['warc_url'], 'licenses': labeled_licenses}, indent=2))
    responses.append(doc_with_labels)

Given the followings HTML snippets enclosed in ```quotes```, respond YES if the Creative Common license mentioned in the snippets refers to to the text content of the web page, otherwise respond NO.

Examples of "NO" include:
- The Creative Common license refers to the images on the page.
- The is another license mentioned on the snippet.
- License is not in an official Creative Common format.

Examples of "YES" include:
- Copyright or ©️ is mentioned on the page AND text content is licensed under Creative Common; it is ok if the page is copyrighted if a Creative Common license is also mentioned.
- All "work" or "content" is mentioned as being Creative Common licensed.
- The Creative Commons tag appears on the footer on the page with no extra content.
- Public domain license is mentioned on the page.


DO NOT return anything other than "YES" or "NO".

```
{snippet}
```


  0%|          | 0/467 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
formatted_content = [['license', 'url', 'predicted', 'label', 'snippet', 'text', 'url', 'id', "source", "model"]]

for page in responses:
    for license_name, license_output in page["metadata"]["labeled_licenses"].items():
        formatted_content.append(
            [
                license_name,
                page["metadata"]["warc_url"],
                license_output["label"],
                "",
                license_output["snippet"],
                page['text'],
                page["metadata"]["warc_url"],
                page["id"],
                page["source"],
                license_output["model"]
            ]
        )

with open('../temp/output-v4.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(formatted_content)

In [47]:
print('YES: ', sum(1 for row in formatted_content if row[2] == "YES"))
print('NO:  ', sum(1 for row in formatted_content if row[2] == "NO"))
print('=' * 40)
for row in formatted_content:
    if row[2] == "NO":
        print(row[0])
        print(row[1])
        loc = row[4].find("creativecommons.org/licenses")
        print(repr(row[4][loc - 50:loc + 50]))
        print(row[-1])
        print('=' * 40)


YES:  101
NO:   28
cc_re__cc_re__cc_by_sa_2_0
https://grandpaguysstories.com/2015/10/07/juarez-mexico/
' Plate Collection) [CC BY-SA 2.0 (<a href="http://creativecommons.org/licenses/by-sa/2.0" rel="nofol'
meta-llama/Llama-3-70b-chat-hf
cc_re__cc_re__cc_by_2_5
http://samm.univ-paris1.fr/CEO-s-Morality-and-Incentives?debut_article_rubrique_numerotes=70
"webhub</a> habillage licence&nbsp;<a href='http://creativecommons.org/licenses/by/2.5/'>Creative Com"
meta-llama/Llama-3-70b-chat-hf
cc_re__cc_re__cc_publicdomain_zero_1_0
http://collection.sciencemuseum.org.uk/objects/co102905/pendant-drop-shaped-yellow-opaque-amber-bead-wi-pendant
'\n              <p>\n              <a href="https://creativecommons.org/licenses/by/4.0/">\n           '
meta-llama/Llama-3-70b-chat-hf
cc_re__cc_re__cc_publicdomain_zero_1_0
https://bmccancer.biomedcentral.com/articles/10.1186/s12885-015-1485-5
'g/1999/xhtml" class="ExternalRef"><a href="http://creativecommons.org/licenses/by/4.0"><span class="'
meta-llama/